In [1]:
from transformation import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
fun = transformation_functions()
x = "The Boots and the Cats.  "
y = "28/10/2006"
list(map(lambda x: x.__name__,fun))
lift = lambda f,l : list(map(f,l))
clean = lambda x: [e for e in x if e]

In [3]:
print(lowercase(x))
print(uppercase(x))

print(strip_whitespace(x))
print(remove_whitespace(x))
print(tokenize(x))
print(lift(uppercase,tokenize(x)))
print(lift(remove_whitespace,tokenize(x)))
print(remove_punctuation(x))
print(lift(uppercase,lift(remove_punctuation,tokenize(x))))
print(lift(stem,tokenize(x)))
print(clean(lift(remove_stopwords,tokenize(x))))


the boots and the cats.  
THE BOOTS AND THE CATS.  
The Boots and the Cats.
TheBootsandtheCats.
['The', 'Boots', 'and', 'the', 'Cats', '.']
['THE', 'BOOTS', 'AND', 'THE', 'CATS', '.']
['The', 'Boots', 'and', 'the', 'Cats', '.']
The Boots and the Cats  
['THE', 'BOOTS', 'AND', 'THE', 'CATS', '']
['the', 'boot', 'and', 'the', 'cat', '.']
['The', 'Boots', 'Cats', '.']


In [7]:
tokenize.__dir__()

['__new__',
 '__repr__',
 '__call__',
 '__get__',
 '__closure__',
 '__doc__',
 '__globals__',
 '__module__',
 '__builtins__',
 '__code__',
 '__defaults__',
 '__kwdefaults__',
 '__annotations__',
 '__dict__',
 '__name__',
 '__qualname__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__init__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [32]:
tokenize("lmmm")

['lmmm']

In [42]:
isinstance(list(map(tokenize,["ko","ko"])),list(list))

TypeError: 'type' object is not iterable

In [4]:
_x = tokenize(x)
print(_x)
uppercase(*_x)

['The', 'Boots', 'and', 'the', 'Cats', '.']


TypeError: uppercase() takes 1 positional argument but 6 were given

In [5]:
nl = []
for f in transformation_functions:
    nl.append(lift(f))

[<function transformation.identity(x)>,
 <function transformation.lowercase(x)>,
 <function transformation.uppercase(x)>,
 <function transformation.strip_whitespace(x)>,
 <function transformation.remove_whitespace(x)>,
 <function transformation.remove_punctuation(text)>,
 <function transformation.stem(text)>,
 <function transformation.remove_stopwords(text)>,
 <function transformation.tokenize(text)>]